# Energy Expenditure

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import seaborn as sns

import Tracab as tracab
import Tracking_Visuals as vis
import Tracking_Velocities as vel
import helpers

import importlib
importlib.reload(tracab)

<module 'Tracab' from '/Users/jeffbalkanski/research_soccer/SoccerTrackingData3/Tracab.py'>

In [2]:
# config
current_dir = os.path.dirname(os.getcwd())
fpath = os.path.join(current_dir, 'TrackingSample') # path to directory of Tracab data
LEAGUE = 'DSL'

## Data

In [3]:
# read frames, match meta data, and data for individual players
fname = '984628'
frames_tb, match_tb, team1_players, team0_players = tracab.read_tracab_match_data(LEAGUE, fpath, fname, verbose=True)

Reading match metadata
Reading match tracking data
Timestamping frames
Measuring velocities
home goalkeeper(s):  [1]
away goalkeeper(s):  [73]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



67616 139808


In [7]:
# pick a player
player = team0_players[5]

In [9]:
# metrics available
target_ex = player.frame_targets[2]
vars(target_ex)

{'team': 0,
 'sys_target_ID': 15,
 'jersey_num': 5,
 'pos_x': -1827.0,
 'pos_y': -666.0,
 'speed': 0.2,
 'timestamp': 0.08,
 'vx': 0.339,
 'vy': 0.071,
 'speed_filter': 0.346,
 'ax': 0.5,
 'ay': 0.25,
 'a_magnitude': 0.559}

# Acceleration

In [8]:
vel.estimate_player_accelerations(team1_players, team0_players, match_tb)


Warning, must get rid of accelerations that are too high > need to find a threshold


In [21]:
velocities = helpers.get_all_values(player, 'speed_filter')
accelerations = helpers.get_all_values(player, 'ax')

# Energy expenditure calculation

We want to make make time series of energy expenditure for each player

1. **VeBVD =** sum of acceleration vectors
2. **metabolic power**  = 5th degree polynomial

In [ ]:
player = team0_players[5]

In [ ]:
player.frame_targets[100].acceleration

## VeBDA - vector body acceleration
the VeDBA at time t, is `a_magnitude`.

For a possession starting at $s_n$ and ending at $e_n$ , the VeBDA for that posession is 
$$V'_n = \sum_{t=s_n}^{e_n}V_t$$

In [10]:
# get for all players
all_players = [x for x in team0_players.items()] + [x for x in team1_players.items()]
for (num, player) in all_players:
    VeBDA =[sum(player.a_magnitude[:i]) for i in range(len(player.a_magnitude))]
    player.VeBDA = VeBDA

AttributeError: 'tracab_player' object has no attribute 'a_magnitude'

In [15]:
for i, player in all_players:
    print(vars(player))
    break

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
NotebookApp.iopub_data_rate_limit

NameError: name 'NotebookApp' is not defined

## Metabolic power
First must find the **energy cost**
$$EC =155.4i^5 −30.4i^4 −43.3i^3 +46.3i^2 +19.5i +3.6$$

In [17]:
import NotebookApp

ModuleNotFoundError: No module named 'NotebookApp'